In [ ]:
import glob
import logging
import os
import pandas as pd

currentDirectory = os.getcwd()

if not os.path.exists('data'):
    while currentDirectory != os.path.dirname(currentDirectory):
        parentDirectory = os.path.dirname(currentDirectory)

        if os.path.exists(os.path.join(parentDirectory, 'data')):
            os.chdir(parentDirectory)
            break

        currentDirectory = parentDirectory

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [ ]:
filePattern = 'data/store_1_*.csv'
receivingStoreInventory = pd.concat([pd.read_csv(f) for f in glob.glob(filePattern)], ignore_index=True)
receivingStoreInventory['Qty.'] = pd.to_numeric(receivingStoreInventory['Qty.'].fillna(0), errors='coerce')
receivingStoreInventory['Price'] = pd.to_numeric(receivingStoreInventory['Price'].fillna(0), errors='coerce')

sendingStoreInventory = pd.read_csv('data/store_2.csv')
sendingStoreInventory['Qty.'] = pd.to_numeric(sendingStoreInventory['Qty.'].fillna(0), errors='coerce')
sendingStoreInventory = sendingStoreInventory.loc[sendingStoreInventory['Qty.'] > 0]

# Data cleaning

In [ ]:
def checkDuplicatesColumnDifferences(df: pd.DataFrame, columnToCheck: str, columnDuplicated: str = 'Item') -> tuple[int, list[str]]:
    sortedDuplicates = df.loc[df.duplicated(subset=[columnDuplicated], keep=False)].sort_values('Item')
    differentPrices = []

    for duplicate in sortedDuplicates[columnDuplicated].unique():
        duplicateData = sortedDuplicates.loc[sortedDuplicates[columnDuplicated] == duplicate]
        uniquePrices = duplicateData[columnToCheck].nunique()

        if uniquePrices > 1:
            differentPrices.append(duplicate)
            
    logging.info(f'Duplicates with different "{columnToCheck}" #{len(differentPrices)}')
    return len(differentPrices), differentPrices


def mergeDuplicates(df: pd.DataFrame, columnsToMerge: dict[str, str], columnDuplicated: list[str] = ['Item']) -> pd.DataFrame:
    logging.info(f'Duplicate item #{df.duplicated(subset=columnDuplicated).sum()}')
    mergedDuplicates = df.groupby(columnDuplicated).agg(columnsToMerge).reset_index()
    df = pd.merge(df.drop_duplicates(subset=columnDuplicated, keep = "first").drop(columns=list(columnsToMerge.keys())), mergedDuplicates,
                  on=columnDuplicated,
                  how='left')
    
    logging.info(f'Duplicate item #{df.duplicated(subset=columnDuplicated).sum()}')
    return df

In [ ]:
# Sending store
numberOfDuplicatesPrices, differentPrices = checkDuplicatesColumnDifferences(receivingStoreInventory, 'Price')
numberOfDuplicatesUPC, differentUPC = checkDuplicatesColumnDifferences(receivingStoreInventory, 'UPC')
receivingStoreInventory = mergeDuplicates(receivingStoreInventory, {'Qty.': 'sum', "Price": "mean", "UPC": "first"})

# Receiving store
sendingStoreInventory = mergeDuplicates(sendingStoreInventory, {'Qty.': 'sum'})